API Call which gets the images in the results variable

In [11]:
from PIL import Image
import requests
import cv2
import numpy as np
from urllib.request import urlopen

import xml.etree.ElementTree as ET
import flickrapi

api_key = u'efb284edc88f1563e0e398e0d42ca677'
api_secret = u'da3cfff301f5ea38'

#200368767@N08 my user ID

flickr = flickrapi.FlickrAPI(api_key, api_secret)
flickr.authenticate_via_browser(perms='read')

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='etree')
url_type = 'url_n'
results = flickr.photos.search(page='5', tag_mode='all', tags="mountain, alps, snow", extras=url_type, per_page=4, sort = 'relevance', geo_context = '0', content_types = '0')


Support method for Showing an image

In [2]:
def ShowWindow(image, title, i, row) -> int:

    #show image, move it to position, and increment i
    cv2.imshow(f'{title}: {row},{i}', image)
    cv2.moveWindow(f'{title}: {row},{i}', (i%8)*250, row*250)

    return i+1

Unused Testing

In [ ]:
column = 0
for photo in results:

    row = 0
    for data in photo:
        
        title = data.get('title')
        url = data.get(url_type)

        if(url != None):

            image_raw = PostProcesser.GetImageFromUrl(url)
            #decode the image with color
            image = cv2.imdecode(image_raw, cv2.IMREAD_COLOR)
            #resize the image
            image = cv2.resize(image, (250, 250))

            column = ShowWindow(image, "original", column, row) 

            image = PostProcesser.GetNormalizedImg(image)
            #column = ShowWindow(image, "normalized", column, row)

            imageKMeans = cv2.add(PostProcesser.GetKMeanImg(image, num_clusters = 6), 25)
            column = ShowWindow(imageKMeans, "KMEAN", column, row)

            imageSkeleton = PostProcesser.GetSkeletonImg(imageKMeans, kernelSize = 3, _iterations = 1)
            column = ShowWindow(imageSkeleton, "skeleton", column, row)

            denoisedImg = PostProcesser.GetDenoisedImg(imageKMeans)
            column = ShowWindow(denoisedImg, "denoised", column, row)   

            imageDeNoisedGray = PostProcesser.GetGrayscaleResizedImg(denoisedImg)
            #column = ShowWindow(imageDeNoisedGray, "grayscaled", column, row)        

            imageGraySkeleton = PostProcesser.GetSkeletonImg(imageDeNoisedGray, kernelSize = 5, _iterations = 1)
            column = ShowWindow(imageGraySkeleton, "graySkel", column, row)     
            
            imageGraySkeletonBlurred = PostProcesser.GetBlurredImg(imageGraySkeleton, 19)
            #column = ShowWindow(imageGraySkeletonBlurred, "graySkelBlur", column, row)   

            processedImage = cv2.cvtColor(imageGraySkeletonBlurred, cv2.COLOR_GRAY2BGR)
            processedImage = cv2.drawContours(processedImage, PostProcesser.GetContours(processedImage, False, threshold1 = 30, threshold2 = 120), -1, (0, 255, 255), 1)
            column = ShowWindow(processedImage, "processed1", column, row) 
            
            processedImage2 = cv2.cvtColor(PostProcesser.GetBlurredImg(imageDeNoisedGray, 19), cv2.COLOR_GRAY2BGR)
            processedImage2 = cv2.drawContours(processedImage2, PostProcesser.GetContours(processedImage2, False, threshold1 = 30, threshold2 = 120), -1, (0, 0, 255), 1)
            column = ShowWindow(processedImage2, "processed2", column, row) 
            



            #cv2.bitwise_or(image, postProcesser.GetEdges(image_raw), image)
            #processedImage = cv2.drawContours(image, postProcesser.GetContours(image, False), -1, (0, 0, 255), 1)
            #morph = cv2.drawContours(image, postProcesser.GetContours(image, True), -1, (0, 255, 0), 1)
            
        print()
        row += 1
        column = 0

cv2.waitKey(0)
cv2.destroyAllWindows()

Prosessing of a single image (this is where the magic happens):

In [3]:
import cv2
from cv2.typing import MatLike, Point, Size
import numpy as np
from Processer import PostProcesser

def ProcessAndShow(image:MatLike, column, row):
    
    column = ShowWindow(image, "original", column, row) 

    gray = PostProcesser.GetGrayscaleResizedImg(image)  
    imageDeNoised = PostProcesser.GetDenoisedImg(image)
    blurred = PostProcesser.GetBlurredImg(image, 15)
    blurredDeNoised = PostProcesser.GetBlurredImg(imageDeNoised, 15)

    threshold1 = 35
    threshold2 = 100
    
    processedImage = PostProcesser.GetContours(blurred, blurred, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage, "P_Blurred", column, row) 

    processedImage2 = PostProcesser.GetContours(imageDeNoised, imageDeNoised, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage2, "P_Denoised", column, row) 

    processedImage3 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, True, threshold1, threshold2, 2)
    column = ShowWindow(processedImage3, "P_blurredDeN", column, row) 
    
    processedImage = PostProcesser.GetContours(blurred, blurred, False, threshold1, threshold2, 2)
    column = ShowWindow(processedImage, "P_Blurred2", column, row) 

    #processedImage2 = PostProcesser.GetContours(imageDeNoised, imageDeNoised, False, 20, 200, 2)
    #column = ShowWindow(processedImage2, "P_Denoised2", column, row) 
    
    #processedImage = postProcesser.GetBinaryContoured(image, gray, thresholded)
    processedImage3 = PostProcesser.GetContours(blurredDeNoised, blurredDeNoised, False, threshold1, threshold2, 2)
    column = ShowWindow(processedImage3, "P_blurredDen2", column, row) 

Main method:

In [12]:
from Processer import PostProcesser

#print("result length", len(results[0]))

try:
    if(len(results[0]) > 0):
        column = 0
        for photo in results:
            row = 0
            for data in photo:
                title = data.get('title')
                url = data.get(url_type)

                if(url != None):

                    image_raw = PostProcesser.GetImageFromUrl(url)
                    #decode the image with color
                    image = cv2.imdecode(image_raw, cv2.IMREAD_COLOR)
                    #resize the image
                    image = cv2.resize(image, (250, 250))

                    ProcessAndShow(image, column, row)

                    row += 1
                    column = 0
    else:
        image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
        ProcessAndShow(image, 0, 0)   
except:
    image = PostProcesser.GetImageFromPath("C:\\mountain1.jpg")
    ProcessAndShow(image, 0, 0)

cv2.waitKey(0)
cv2.destroyAllWindows()

INDEX   59896 shape:  (59896, 2) WHICH ARE:  [[  0   0]
 [  0   1]
 [  0   2]
 ...
 [249 247]
 [249 248]
 [249 249]]  so first element is:  (0, 0)
ORIG SHAPE (250, 250, 3)  MASK SHAPE:  (252, 252)
mask  [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 1]]
INDEX   26792 shape:  (26792, 2) WHICH ARE:  [[ 64  99]
 [ 65  99]
 [ 65 100]
 ...
 [249 247]
 [249 248]
 [249 249]]  so first element is:  (64, 99)
ORIG SHAPE (250, 250, 3)  MASK SHAPE:  (252, 252)
mask  [[1 1 1 ... 1 1 1]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
INDEX   262 shape:  (262, 2) WHICH ARE:  [[ 64  99]
 [ 65  99]
 [ 65 100]
 [ 65 101]
 [ 65 102]
 [ 66  99]
 [ 66 100]
 [ 66 101]
 [ 66 102]
 [ 66 103]
 [ 67  99]
 [ 67 100]
 [ 67 101]
 [ 67 102]
 [ 67 103]
 [ 67 104]
 [ 68  98]
 [ 68  99]
 [ 68 100]
 [ 68 101]
 [ 68 102]
 [ 68 103]
 [ 68 104]
 [ 69  98]
 [ 69  99]
 [ 69 100]
 [ 69 101]
 [ 69 102]
 [ 